# Covid19 Greece Analysis

## Install the dependencies

In [9]:
!pip install pandas bokeh numpy scipy folium ipywidgets_bokeh

## Get an overview of the situation

In [19]:
import pandas as pd
from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.io import show, output_notebook
from bokeh.layouts import row

output_notebook()

names = ["city", "latitude", "longitude", "infections"]
df = pd.read_csv(
    'cities.csv',
    names=names,
)

data = df.loc[:, ['city', 'infections']]
data = data.sort_values(ascending=True, by=['infections'])

p = figure(x_range=data['city'], title="Μολύνσεις ανά πόλη",
           x_axis_label="Πόλη", y_axis_label="Αριθμός Ανθρώπων", height = 320, sizing_mode = 'scale_width')

p.vbar(x=data['city'], top=data['infections'], width=0.9)
p.tools = [] # disable zoom / pan
p.add_tools(HoverTool(tooltips=[("City", "@x"), ("Infected", "@top")]))
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.toolbar.logo = None
p.toolbar_location = None


show(p)

Loading BokehJS ...

## Visualize in a Map

In [11]:
import folium

MAGNIFICATION = 100

# map centered at Athens
map = folium.Map(
    location=[37.9838, 23.7275],
#     width=500,
    height=600,
)

for i, city in df.dropna().iterrows():
    # add a tooltip showing the number of infections on top
    popup = folium.Popup(str(city["infections"]), sticky=True, show=True)
    folium.Circle(
        location=[
            city["latitude"],
            city["longitude"]
        ],
        radius=city["infections"] * MAGNIFICATION,
        color="red",
        sticky=True,
        show=True,
        fill=True,  # Set fill to True
        fill_opacity=0.7
    ).add_child(popup).add_to(map)

sw = df[['latitude', 'longitude']].min().values.tolist()
ne = df[['latitude', 'longitude']].max().values.tolist()
map.fit_bounds([sw, ne])

map

# Predictions

We fit the data to a Sigmoid curve

In [12]:
from bokeh.models import ColumnDataSource

# load the data
names = ["date", "infections"]
ts = pd.read_csv(
    'greece.csv',
    names=names,
)
ts['date'] = pd.to_datetime(ts['date'], dayfirst=True)
ts['total'] = ts['infections'].cumsum()

# infections graph
p = figure(
    title = "Κρούσματα", 
    x_axis_label = "Date", 
    y_axis_label = "Αριθμός Ανθρώπων", 
    x_axis_type = "datetime",
    height = 320, 
    sizing_mode = 'scale_width'
)
p.tools = []

def hover(renderer, tooltips):
    hover = HoverTool(
        renderers = [renderer],
        mode = "vline",
        tooltips = tooltips,
        formatters={"date": "datetime"}
    )
    return hover

source = ColumnDataSource(data = ts)
per_day = p.line(x="date", y="infections", source=source, legend_label = 'Κρούσματα ανά ημέρα')
total = p.line(x="date", y="total", source=source, color = 'green', legend_label = 'Συνολικά Κρούσματα')

p.add_tools(hover(per_day, [("Date", "@date"), ("New infections", "@infections")]))
p.add_tools(hover(total, [("Total Infections", "@total")]))

# show the legend in a nice place
p.legend.location = "top_left"
p.legend.click_policy="hide"

show(p)

# Predictions and Curve Fitting to a Logistic Curve

In [14]:
from scipy.optimize import curve_fit
from bokeh.io import push_notebook
from bokeh.models import Span
import datetime
from ipywidgets import interact, SelectionSlider, widgets
import numpy as np

# f(x) = \frac{L}{1 + e^{-k(x-x0)}}
def sigmoid(x, L, x0, k):
    y = L / (1 + np.exp(-k*(x-x0)))
    return y

def sigmoid_with_growth(growth):
    return lambda x, L, x0: sigmoid(x, L, x0, growth)


def sigmoid_inv(y, L, x0, k):
    x = -(np.log(L/y - 1) / k) + x0
    return x
first_infection = ts['date'].loc[0]
SAMPLES = 15

# get days since first infection
x_train = (ts['date'] - first_infection).dt.days[:SAMPLES]
y_train = list(ts['total'])[:SAMPLES]


    
plot = figure(y_range=(0,7000), title = "Predictions", x_axis_label = "Date", y_axis_label = "Total Infections", x_axis_type="datetime", height = 320, sizing_mode = 'scale_width')


# try to project 90 days since the first incident
real = plot.asterisk(ts['date'][:SAMPLES], ts['total'][:SAMPLES])
pred = plot.circle_x(ts['date'][SAMPLES:], ts['total'][SAMPLES:], line_color="#f0027f")
plot.xaxis.fixed_location = 0
hover = HoverTool(
    mode="vline",
    tooltips=[("date", "@x"), ("Estimated infections", "@y{0}")], 
    formatters={'@x': 'datetime'}
)
# plot.tools = []
plot.add_tools(hover)

hover_item = HoverTool(
    renderers = [real, pred],
    tooltips=[("date", "@x"), ("Measured infections", "@y{0}")], 
    formatters={'@x': 'datetime'}
)
plot.add_tools(hover_item)

x = range(0, 60) # 90 days forecast
x_days = first_infection + pd.to_timedelta(x, unit = 'D')
growth_rate = 0.23
popt, pcov = curve_fit(sigmoid_with_growth(growth_rate), x_train, y_train, maxfev=10000)
y = sigmoid(x, *popt, growth_rate)
# draw lines at the sigmoid's 
curve = plot.line(x_days, y)
ten_pct = 0.1 * y[-1]
ninety_pct = 0.9 * y[-1]
x_ten_pct = sigmoid_inv(ten_pct, *popt, growth_rate)
x_ninety_pct = sigmoid_inv(ninety_pct, *popt, growth_rate)

ten_pct_line = Span(
    location=first_infection + datetime.timedelta(days = x_ten_pct), 
    dimension='height', 
    line_color='red', 
    line_width=1,
    line_dash='dotted',
)

ninety_pct_line = Span(
    location=first_infection + datetime.timedelta(days = x_ninety_pct), 
    dimension='height', 
    line_color='red', 
    line_width=1,
    line_dash='dotted',
)

plot.renderers.extend([ten_pct_line, ninety_pct_line])
ten_pct_id = len(plot.renderers) - 2
ninety_pct_id = len(plot.renderers) -1

# update the vertical lines which indicate the lifetime of the curve
# i.e. the 10-90% bounds
def render_lifecycle(popt, growth_rate):
    y = popt[0]
    print("Maximum infections", int(y))
    
    # get 10%-90% bounds
    ten_pct = 0.1 * y
    ninety_pct = 0.9 * y
    x_ten_pct = sigmoid_inv(ten_pct, *popt, growth_rate)
    x_ninety_pct = sigmoid_inv(ninety_pct, *popt, growth_rate)
    print("Lifecycle", x_ninety_pct - x_ten_pct)
    
    plot.renderers[ten_pct_id].location = first_infection + datetime.timedelta(days = x_ten_pct)
    plot.renderers[ninety_pct_id].location = first_infection + datetime.timedelta(days = x_ninety_pct)

    
# create an interactive plot
def render(k=0.235, SAMPLES = 15):
    x_train = (ts['date'] - first_infection).dt.days[:SAMPLES]
    y_train = list(ts['total'])[:SAMPLES]
    
    # update the trained values
    real.data_source.data = {'x': ts['date'][:SAMPLES], 'y':  ts['total'][:SAMPLES] }
    
    # update the new values
    pred.data_source.data = {'x': ts['date'][SAMPLES:], 'y': ts['total'][SAMPLES:] }
    
    popt, pcov = curve_fit(sigmoid_with_growth(k), x_train, y_train, maxfev=10000)
    y = sigmoid(x, *popt, k)
    curve.data_source.data['y'] = y
    render_lifecycle(popt, k)
    push_notebook()


show(plot, notebook_handle=True)

k = widgets.FloatSlider(value=0.235, min=0.225, max=0.27, step=0.001, readout=True, readout_format='.4f')
interact(render, k=k, samples = (10, len(ts)))

interactive(children=(FloatSlider(value=0.235, description='k', max=0.27, min=0.225, readout_format='.4f', ste…

<function __main__.render(k=0.235, SAMPLES=15)>